In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.pluginbase

In [ ]:
# export
from pymemri.data.schema import *
from pymemri.pod.client import *
from pymemri.imports import *
from pymemri.pod.utils import *
from os import environ
from abc import ABCMeta
import abc
import json
import importlib
import string
import time
from enum import Enum
from fastscript import *
import os
from pymemri.plugin.schema import PluginRun
from pymemri.data.basic import *

In [ ]:
# hide
from nbdev.showdoc import *

# Plugins

PluginBase is the plugin class that the simplest plugin inherits.

Inheriting class should implement:

- run()           that implements the logic of the plugin
- add_to_schema() for plugin specific item types

In [ ]:
# export
POD_FULL_ADDRESS_ENV        = 'POD_FULL_ADDRESS'
POD_TARGET_ITEM_ENV         = 'POD_TARGET_ITEM'
POD_OWNER_KEY_ENV           = 'POD_OWNER'
POD_AUTH_JSON_ENV           = 'POD_AUTH_JSON'

In [ ]:
# export
# hide
class PluginBase(Item, metaclass=ABCMeta):
    """Base class for plugins"""
    properties = Item.properties + ["name", "repository", "icon", "query", "bundleImage",
                                    "runDestination", "pluginClass"]
    edges = Item.edges

    def __init__(self, client=None, run_id=None, name=None, repository=None, icon=None,
                 query=None, bundleImage=None, runDestination=None, pluginClass=None, **kwargs):
        if pluginClass is None: pluginClass=self.__class__.__name__
        super().__init__(**kwargs)
        self.client = client
        self.run_id = run_id
        self.name = name
        self.repository = repository
        self.icon = icon
        self.query = query
        self.bundleImage = bundleImage
        self.runDestination = runDestination
        self.pluginClass = pluginClass
        
    def get_run(self):
        return self.client.get(self.run_id, expanded=False)
    
    def get_state(self):
        return self.get_run().state
        
    def get_account(self):
        run = self.get_run()
        return run.account[0] if len(run.account) > 0 else None
    
    def get_settings(self):
        run = self.get_run(self.client)
        return json.loads(run.settings)
        
    def set_vars(self, vars):
        run = self.get_run()
        for k,v in vars.items():
            if hasattr(run, k):
                setattr(run, k, v)
        run.update(self.client)
        
    def set_state(self, state, message=None):
        self.set_vars({'state': state, 'message': message})
        
    def set_account(self, account):
        existing = self.get_account()
        if existing:
            account.id = existing.id
            account.update(self.client)
        else:
            run = self.get_run()
            run.add_edge('account', account)
            run.update(self.client)
        
    def set_settings(self, settings):
        self.set_vars({'settings': json.dumps(settings)})
        
    @abc.abstractmethod
    def run(self):
        raise NotImplementedError()

    @abc.abstractmethod
    def add_to_schema(self):
        raise NotImplementedError()

## Creating a plugin

The memri [pod](https://gitlab.memri.io/memri/pod) uses a plugin system to add features to the backend memri backend. Plugins can import your data (importers), change your data (indexers), or call other serivces. Users can define their own plugins to add new behaviour to their memri app. Let's use the following plugin as an example of how we can start plugins.

In [ ]:
# export
# hide
class MyItem(Item):
    properties = Item.properties + ["name", "age"]
    edges = Item.edges
    def __init__(self, name=None, age=None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.age = age

class MyPlugin(PluginBase):
    """"""
    properties = PluginBase.properties + ["containerImage"]
    edges= PluginBase.edges

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run(self):
        print("running")
        self.client.create(MyItem("some person", 20))

    def add_to_schema(self):
        self.client.add_to_schema(MyItem("my name", 10))

In [ ]:
# hide
MyPlugin()

MyPlugin (#None)

```python
class MyItem(Item):
    properties = Item.properties + ["name", "age"]
    edges = Item.edges
    def __init__(self, name=None, age=None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.age = age

class MyPlugin(PluginBase):
    """"""
    properties = PluginBase.properties
    edges= PluginBase.edges
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def run(self):
        print("running")
        self.client.create(MyItem("some person", 20))
        
    def add_to_schema(self):
        self.client.add_to_schema(MyItem("my name", 10)) 
```

Memri plugins need to define at least 2 methods: `.run()` and `.add_to_schema()`. `.run()` defines the logic of the plugin. `.add_to_schema()` defines the schema for the plugin in the pod. Note that currently, `add_to_schema` requires all item to **have all properties defined that are used in the plugin**. In the future, we might replace add_to_schema, to be done automatically, based on a declarative schema defined in the plugin. 

## Helper classes -

In [ ]:
# hide
# export
def get_plugin_cls(plugin_module, plugin_name):
    try:
        module = importlib.import_module(plugin_module)
        plugin_cls = getattr(module, plugin_name)
        return plugin_cls
    except (ImportError, AttributeError):
        raise ImportError(f"Unknown plugin: {plugin_module}.{plugin_name}")

def run_plugin_from_run_id(run_id, client):
    """
    Args:
        run_id (int): id of the PluginRun
        client (PodClient): client containing PluginRun
        return_plugin (bool): Returns created plugin instance for testing purposes.
    """

    run = client.get(run_id)
    plugin_cls = get_plugin_cls(run.pluginModule, run.pluginName)
    plugin = plugin_cls(client=client, run_id=run_id)
    plugin.add_to_schema()

    # TODO handle plugin status before run
    plugin.run()

    return plugin

In [ ]:
# export
# hide
def register_base_schemas(client):
    try:
        assert client.add_to_schema(PluginRun("", "", "", state="", message="", targetItemId=""))
        assert client.add_to_schema(Account(service="", identifier="", secret=""))
    except Exception as e:
        raise ValueError("Could not add base schema")

### Test Plugin Runtime

In [ ]:
# hide
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account
from pymemri.plugin.pluginbase import PluginRun, register_base_schemas, run_plugin_from_run_id

client = PodClient()
register_base_schemas(client)

# Create a dummy account to use for authentication within the plugin
account = Account(service="my_plugin_service", identifier="username", secret="password")
account.update(client)

# Create a run to enable plugin runtime
run = PluginRun("pymemri", "pymemri.plugin.pluginbase", "MyPlugin", account=[account])
run.update(client)

plugin = run_plugin_from_run_id(run.id, client)

# check if account is accessible
assert plugin.get_account().identifier == "username"

# set a state
plugin.set_state("test_state", message="test_message")
assert plugin.get_state() == "test_state"


## Run from id test -

In [ ]:
# hide
# skip
client = PodClient()
run = PluginRun("pymemri", "pymemri.plugin.pluginbase", "MyPlugin", state="not started")
account = Account(identifier="login", secret="password")
run.add_edge("account", account)
assert client.add_to_schema(PluginRun("", "", "", "", ""))
assert client.create(run)
assert client.create(account)
assert client.create_edge(run.get_edges("account")[0])
run = client.get(run.id)
run_plugin_from_run_id(run.id, client);

running
logging in with account login and password password


In [ ]:
# export
# hide
def _parse_env():
    env = os.environ
    print("Reading `run_plugin()` parameters from environment variables")
    try:
        pod_full_address = env.get(POD_FULL_ADDRESS_ENV, DEFAULT_POD_ADDRESS)
        plugin_run_json  = json.loads(str(env[POD_TARGET_ITEM_ENV]))
        print(plugin_run_json)
        plugin_run_id    = plugin_run_json["id"]
        owner_key        = env.get(POD_OWNER_KEY_ENV)
        pod_auth_json    = json.loads(str(env.get(POD_AUTH_JSON_ENV)))
        return pod_full_address, plugin_run_id, pod_auth_json, owner_key
    except KeyError as e:
        raise Exception('Missing parameter: {}'.format(e)) from None

## Running your plugin using the CLI

Plugins can be started using the pymemri `run_plugin` or `simulate_run_plugin_from_frontend` CLI. With `run_plugin` the plugin is invoked directly by spawning a new python process, while `simulate_run_plugin_from_frontend` requests the pod to spawn a new process, docker container, or kubernetes container, which in calls `run_plugin` (for more info see `simulate_run_plugin_from_frontend`. When using `run_plugin`, you can either pass your run arguments as parameters, or set them as environment variables. If both are set, the CLI will use the passed arguments.

In [ ]:
# export
# hide
@call_parse
def store_keys(path:Param("path to store the keys", str)=DEFAULT_POD_KEY_PATH,
               database_key:Param("Database key of the pod", str)=None,
               owner_key:Param("Owner key of the pod", str)=None):
    
    if database_key is None: database_key = PodClient.generate_random_key()
    if owner_key is None: owner_key = PodClient.generate_random_key()

    obj = {"database_key": database_key,
           "owner_key": owner_key}
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        timestr = time.strftime("%Y%m%d-%H%M%S")
        path.rename(POD_KEYS_FULL_FOLDER / f"keys-{timestr}.json")
    write_json(obj, path)

In [ ]:
# hide
store_keys()

In [ ]:
# export
# hide
def parse_config(file, remove_container=False):
    json_dict = read_json(file)
    account = Account.from_json(json_dict["account"])
    del json_dict["account"]
    settings = json.dumps(json_dict["settings"])
    del json_dict["settings"]
    run = PluginRun.from_json(json_dict)
    run.settings = settings
    run.add_edge("account", account)
    if remove_container:
        run.containerImage = "none"
    return run
    
def create_run_expanded(client, run):
    client.create(run)
    accounts = run.account
    if accounts:
        account=accounts[0]
        client.create(account)
        client.create_edge(run.get_edges("account")[0])    

In [ ]:
# export
@call_parse
def run_plugin(pod_full_address:Param("The pod full address", str)=DEFAULT_POD_ADDRESS,
               plugin_run_id:Param("Run id of the plugin to be executed", str)=None,
               database_key:Param("Database key of the pod", str)=None,
               owner_key:Param("Owner key of the pod", str)=None,
               read_args_from_env:Param("Read the args from the environment", bool)=False,
               config_file:Param("config file for the PluginRun", str)=None):
    
    if read_args_from_env:
        pod_full_address, plugin_run_id, pod_auth_json, owner_key = _parse_env()
        database_key=None
    else:
        if database_key is None: database_key = read_pod_key("database_key")
        if owner_key is None: owner_key = read_pod_key("owner_key")
        pod_auth_json = None

    client = PodClient(url=pod_full_address, database_key=database_key, owner_key=owner_key,
                       auth_json=pod_auth_json)
    
    if config_file is not None:
        run = parse_config(config_file, remove_container=True)
        create_run_expanded(client, run)
        plugin_run_id=run.id
    
    print(f"pod_full_address={pod_full_address}\nowner_key={owner_key}\n")

    run_plugin_from_run_id(run_id=plugin_run_id, client=client)

To start a plugin on your local machine, you can use the CLI. This will create a client for you, and run the code defined in `<myplugin>.run()`

In [ ]:
run_plugin(config_file="../example_config.json")

reading database_key from /Users/koen/.pymemri/pod_keys/keys.json
reading owner_key from /Users/koen/.pymemri/pod_keys/keys.json
pod_full_address=http://localhost:3030
owner_key=8677996173299944016776856689234978705539171358808241281853139489

running
logging in with account myusername and password mypassword


> Note: The data that is created here should be in the pod in order for this to work

## Run from pod 

In production, we start plugins by making an API call to the pod, which in turn creates an environment for the plugin and starts it using docker containers, kubernetes containers or a shell script. We can start this process using the `simulate_run_plugin_from_frontend` CLI. **Note that when using docker, provided container name should be "installed" within the Pod environemnt (e.g. `docker build -t pymemri .` for this repo) in order to start it.** 

![running a plugin](images/running_a_plugin.svg)

In [ ]:
# export
from fastcore.script import call_parse, Param
import os

@call_parse
def simulate_run_plugin_from_frontend(pod_full_address:Param("The pod full address", str)=DEFAULT_POD_ADDRESS,
                        database_key:Param("Database key of the pod", str)=None,
                        owner_key:Param("Owner key of the pod", str)=None,
                        container:Param("Pod container to run frod", str)=None,
                        plugin_path:Param("Plugin path", str)=None,
                        account_id:Param("Account id to be used inside the plugin", str)=None,
                        settings_file:Param("Plugin settings (json)", str)=None):
    # TODO remove container, plugin_module, plugin_name and move to Plugin item.
    # Open question: This presumes Plugin item is already in pod before simulate_run_plugin_from_frontend is called.
    if database_key is None: database_key = read_pod_key("database_key")
    if owner_key is None: owner_key = read_pod_key("owner_key")
    params = [pod_full_address, database_key, owner_key]
        
    if (None in params):
        raise ValueError(f"Defined some params to run indexer, but not all")
    client = PodClient(url=pod_full_address, database_key=database_key, owner_key=owner_key)
    for name, val in [("pod_full_address", pod_full_address), ("owner_key", owner_key)]:
        print(f"{name}={val}")

    if settings_file is not None:
        with open(settings_file, 'r') as f:
            settings = f.read()
    else:
        settings = None
        
    if account_id is None:
        account = None
    else:
        account = client.get(account_id)
        print(f"Using {account}")

    register_base_schemas(client)
    run = PluginRun(container, plugin_module, plugin_name, account=[account])

    else:
        if container is None:
            container = plugin_path.split(".", 1)[0]
        print(f"Inferred '{container}' as plugin container name")
        plugin_module, plugin_name = plugin_path.rsplit(".", 1)
        run = PluginRun(container, plugin_module, plugin_name)
        if plugin_id is not None:
            persistent_state = client.get(plugin_id)
            run.add_edge("persistentState", persistent_state)
            client.create_edge(run.get_edges("persistentState")[0])
            client.create(run)
    print(f"\ncalling the `create` api on {pod_full_address} to make your Pod start "
          f"a plugin with id {run.id}.")
    print(f"*Check the pod log/console for debug output.*")
    return run

In [ ]:
client = PodClient()

In [ ]:
!simulate_run_plugin_from_frontend --config_file="../example_config.json"

reading database_key from /Users/koen/.pymemri/pod_keys/keys.json
reading owner_key from /Users/koen/.pymemri/pod_keys/keys.json
pod_full_address=http://localhost:3030
owner_key=8677996173299944016776856689234978705539171358808241281853139489

calling the `create` api on http://localhost:3030 to make your Pod start a plugin with id 4DfDBbcdcbABcf822Fc7DC5d5b3C3BDc.
*Check the pod log/console for debug output.*


In [ ]:
# hide
# !simulate_run_plugin_from_frontend --plugin_path="pymemri.plugin.pluginbase.MyPlugin"

## Appendix -

In [ ]:
# hide
# client.start_plugin("pymemri", run.id)

In [ ]:
# hide
# class StartPlugin(Item):
#     properties = Item.properties + ["container", "targetItemId"]
#     edges = Item.edges
#     def __init__(self, container=None, targetItemId=None, **kwargs):
#         super().__init__(**kwargs)
#         self.container = container
#         self.targetItemId = targetItemId

In [ ]:
# hide
# class PluginSettings(Item):
#     def __init__(self, settings_dict):
#         self.settings_dict=settings_dict
        
#     def __getattr__(self, k):
#         if k in self.settings_dict:
#             return self.settings_dict[k]
        
#     @classmethod
#     def from_string(cls, s):
#         objs = json.loads(s)
#         cls(objs)

In [ ]:
# hide
# # export
# def generate_test_env(client, indexer_run):
#     payload = json.dumps({DATABASE_KEY_ENV: client.database_key, OWNER_KEY_ENV: client.owner_key})
              
#     return {POD_FULL_ADDRESS_ENV: DEFAULT_POD_ADDRESS,
#             POD_TARGET_ITEM: indexer_run.id,
#             POD_SERVICE_PAYLOAD_ENV: payload}

In [ ]:
# hide
# run_plugin(env=generate_test_env(client, run))

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted data.photo.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.pluginbase.ipynb.
Converted pod.client.ipynb.
